<a href="https://colab.research.google.com/github/J0MT/cebra-decoding-miller-dataset/blob/main/CEBRA_figure_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Install the required packages

> And import libraries


In [ ]:
!pip install lazypredict scikit-learn==1.2.2 cebra PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9

In [ ]:
import numpy as np
import os, requests
import numpy as np
from scipy.signal import butter, filtfilt
import pywt
import umap
from sklearn.decomposition import PCA
from cebra import CEBRA

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#@title Installables
!pip install mne
import mne
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
#FOOOF
!pip install fooof
import fooof
from mne.time_frequency import tfr_morlet
from scipy import stats
from scipy.linalg import eig
from scipy.signal import welch
# import sys
# sys.path.insert(0,'/content/drive/Shareddrives/Impact Scholars /Computational Neuroscience/Akito Adaptive Modeling/Program Components')
import matplotlib.gridspec as gridspec
import pandas as pd
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 3.8 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/d

ModuleNotFoundError: No module named 'fooof'

# Load the data as a set of voltages + preprocessing
- 7 patients
- 2 experiments
- 60 trial per experiment
- 3000 fixed trial length
- (46,48,64) channels : we chose only the first 46 channels

The data should have the shape [7,2,60,3000,46]
flatten it to have the shape

> Preprocessing is done via apply a band pass filter and then a wavelet transform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/19_subject_dataset
!ls

Mounted at /content/drive
/content/drive/MyDrive/19_subject_dataset
data  loc


In [ ]:
# Define the paths to the data folder and the locs folder
data_folder = '/content/drive/MyDrive/19_subject_dataset/data'
locs_folder = '/content/drive/MyDrive/19_subject_dataset/loc'

n_channel = 46
limit = 46

subjects_trials = {}

for data_file_name in os.listdir(data_folder)[:19]:
    if data_file_name.endswith('.mat'):
        data_prefix = data_file_name[:2]

        mat = scipy.io.loadmat(os.path.join(data_folder, data_file_name))

        data = mat['data']
        stim_ids = mat['stim'].flatten()

        stim_mask = np.isin(stim_ids, [11, 12])
        transitions = np.diff(stim_mask.astype(int))

        start_indices = np.where(transitions == 1)[0] + 1
        end_indices = np.where(transitions == -1)[0] + 1

        if stim_mask[0]:
            start_indices = np.insert(start_indices, 0, 0)
        if stim_mask[-1]:
            end_indices = np.append(end_indices, len(stim_ids))

        trials = []
        for start, end in zip(start_indices, end_indices):
            trial_data = data[start:end]
            if trial_data.shape[0]!=3000:
                continue
            if limit is not None:
                if trial_data.shape[1] <= limit:
                    continue
            trial_data = trial_data[:,:n_channel]
            trial_stim_ids = stim_ids[start:end]
            trials.append({
                # 'start': start,
                # 'end': end,
                'data': trial_data,
                'stim_ids': [int(p) for p in (np.array(trial_stim_ids) == 11)]
            })
        if len(trials)!=0:
            subjects_trials[data_file_name] = trials
            print(trials[0]['data'].shape)
            print(f'Found {len(trials)} trials for stim_ids 11 and 12.')

(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 30 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 38 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.
(3000, 46)
Found 60 trials for stim_ids 11 and 12.


In [ ]:
X = []
Y = []

for keys in subjects_trials:
    sample_x = []
    sample_y = []
    trials = subjects_trials[keys]
    for j in range(len(trials)):
        sample_x.append(np.array(trials[j]['data']))
        sample_y.append(np.array(trials[j]['stim_ids']))

    X.append(np.array(sample_x))
    Y.append(np.array(sample_y))

print(f'data shape is : {len(X)}')
print(f'Labels shape is : {len(Y)}')

data shape is : 16
Labels shape is : 16


In [ ]:
print(X[0].shape) # example the first patient data shape
print(Y[0].shape) # example the first patient labels shape
print(Y[0][0,:]) # example the first patient the first trial, labels accross all time points .

(60, 3000, 46)
(60, 3000)
[0 0 0 ... 0 0 0]


In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

def bandpower(data, lowcut, highcut, fs=1000):
    """Compute band power for a specific frequency band using a Butterworth filter."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band')  # 4th order bandpass filter

    # Apply filter to each electrode
    filtered_data = filtfilt(b, a, data, axis=0)  # Shape remains [time, electrodes]

    # Compute power (mean squared value)
    power = np.mean(filtered_data ** 2, axis=0)  # Shape: [electrodes]

    return power  # Returns (46,)

def extract_bandpower_features(data, channels=np.arange(46), fs=1000):
    """Extract band power features from ECoG data for Alpha, Beta, Gamma, and High-Gamma bands."""
    alpha_power = bandpower(data, 8, 12, fs)    # Alpha band (8-12 Hz)
    beta_power = bandpower(data, 13, 30, fs)    # Beta band (13-30 Hz)
    gamma_power = bandpower(data, 30, 70, fs)   # Gamma band (30-70 Hz)
    # high_gamma_power = bandpower(data, 70, 150, fs)  # High-Gamma (70-150 Hz)

    # Stack into shape [46, 4] → Flatten to [184]
    feature_vector = np.hstack([alpha_power, beta_power, gamma_power])
    # feature_vector = np.hstack([alpha_power, beta_power, gamma_power, high_gamma_power])
    return feature_vector  # Shape: (184,)

## Creating figures
 ...

In [ ]:
from cebra import CEBRA
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.express as px
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
accuracy_scores = []
n_patients = len(X)
for i in range(n_patients):
    print(f"Testing on patient {i + 1}")
    X_Test = X[i] #(60, 3000, 46)
    Y_Test = Y[i] #(60, 3000)
    X_Train = [] #(17, 60, 3000, 46)
    Y_Train = [] #(17, 60, 3000)
    for j in range(n_patients):
        if i!=j:
            X_Train.append(X[j])
            Y_Train.append(Y[j])
    processed_X_Train = []
    processed_Y_Train = []
    for k in range(len(X_Train)):
        subject = X_Train[k] #(60, 3000, 46)
        for l in range(len(subject)):
            data = subject[l] # (3000,46)
            #process, extract and flattened alpha, beta, gamma powerband for each electrode
            data = extract_bandpower_features(data) #(46*3)
            processed_X_Train.append(data) #(17*60, 46*3)
            processed_Y_Train.append(Y_Train[k][l][0]) #(17*60)
    processed_X_Test = []
    processed_Y_Test = []
    for m in range(len(X_Test)):
        data1 = X_Test[m]
        data1 = extract_bandpower_features(data1) #(46*3)
        processed_X_Test.append(data1) #(60, 46*3)
        processed_Y_Test.append(Y_Test[m][0]) #(60)
    processed_X_Train = np.array(processed_X_Train)
    processed_Y_Train = np.array(processed_Y_Train)
    processed_X_Test = np.array(processed_X_Test)
    processed_Y_Test = np.array(processed_Y_Test)

    max_iterations = 10000
    output_dimension = 3
    cebra_model = CEBRA(model_architecture='offset10-model',
                        batch_size=256,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=output_dimension,
                        max_iterations=max_iterations,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

    # cebra_model.fit(processed_X_Test, processed_Y_Test)  #For creating test embedding
    # test_embed = cebra_model.transform(processed_X_Test)

    cebra_model.fit(processed_X_Train, processed_Y_Train)  #For creating train embedding
    test_embed = cebra_model.transform(processed_X_Train)


# num_samples = cebra_posdir3.shape[0]  # Total number of points
# downsample_size = num_samples // 1 # Reduce dataset by a factor of 10
# random_indices = np.random.choice(num_samples, downsample_size, replace=False) # Randomly select indices

# Downsample the data

    # Assuming test_embed and processed_Y_Test are already defined
    points = test_embed
    labels = processed_Y_Train            #change label to match the set that is being plotted

    # Create a DataFrame
    df = pd.DataFrame(points, columns=["X", "Y", "Z"])
    df["Label"] = labels

    # Define the new color mapping
    color_map = {"0": "blueviolet", "1": "deeppink"}  # Blue-purple and pink

    # Create a dynamic 3D scatter plot
    fig = px.scatter_3d(
        df,
        x="X",
        y="Y",
        z="Z",
        color=df["Label"].astype(str),  # Convert labels to string for discrete colors
        color_discrete_map=color_map,  # Apply the new color mapping
        title="Dynamic 3D Scatter Plot",
        labels={"Label": "Classes"},
        opacity=0.8
    )

    fig.update_traces(marker=dict(size=4))

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-1, 1], tickvals=[-1, -0.5, 0, 0.5, 1]),
            yaxis=dict(range=[-1, 1], tickvals=[-1, -0.5, 0, 0.5, 1]),
            zaxis=dict(range=[-1, 1], tickvals=[-1, -0.5, 0, 0.5, 1]),
            aspectmode="cube"
        )
    )

    # Show the plot
    fig.show()







Testing on patient 1


pos: -0.9857 neg:  5.9721 total:  4.9864 temperature:  1.0000: 100%|██████████| 10000/10000 [01:39<00:00, 100.41it/s]


Testing on patient 2


pos: -0.9797 neg:  5.9762 total:  4.9965 temperature:  1.0000: 100%|██████████| 10000/10000 [01:39<00:00, 100.31it/s]


Testing on patient 3


...

pos: -0.9781 neg:  5.9741 total:  4.9960 temperature:  1.0000: 100%|██████████| 10000/10000 [02:10<00:00, 76.60it/s]


Testing on patient 12


pos: -0.9866 neg:  5.9792 total:  4.9925 temperature:  1.0000: 100%|██████████| 10000/10000 [02:10<00:00, 76.41it/s]


Testing on patient 13


pos: -0.9805 neg:  5.9744 total:  4.9939 temperature:  1.0000:  55%|█████▌    | 5516/10000 [01:12<00:59, 74.94it/s]

In [ ]:
df = mat2dataframe(fname, shift_idx_fields=True)

df = df.drop(columns=["M1_lfp", "PMd_lfp"]) # drop M1_lfp and PMd_lfp columns (lfp is local field potential, and we're not using it here)
# drop M1_lfp_guide and PMd_lfp_guide columns
df = df.drop(columns=["M1_lfp_guide", "PMd_lfp_guide"])
# drop M1_unit_guide and PMd_unit_guide columns
df = df.drop(columns=["M1_unit_guide", "PMd_unit_guide"])
df["trial_id"] = np.arange(len(df)) # pyaldata wants to have trial_id for some fucntions so let's add it
df = df.rename(columns={"tgtDir": "target_direction"}) # rename tgtDir to target_direction
# the data we're using here has bin_size of 30ms (0.03 second), so let's keep it as is
print("original: ", df.M1_spikes[0].shape)
print("combined: ", td.M1_spikes[0].shape)


td = remove_low_firing_neurons(td, "M1_spikes",  5)
td = remove_low_firing_neurons(td, "PMd_spikes", 5)
print("original: ", df.M1_spikes[0].shape)
print("removed:  ", td.M1_spikes[0].shape)

td = transform_signal(td, "M1_spikes",  'sqrt')
td = transform_signal(td, "PMd_spikes", 'sqrt')
td = merge_signals(td, ["M1_spikes", "PMd_spikes"], "both_spikes")
td = add_firing_rates(td, 'smooth')

print("all trials:", td.shape[0])
td = select_trials(td, "result == 'R'")
print("successful trials:", td.shape[0])
move_td = restrict_to_interval(td, "idx_movement_on", end_point_name=None, rel_start=0, rel_end=20)
print("first trial's length in full:", td.both_rates[0].shape)
print("first trial's length in the new dataframe:", move_td.both_rates[0].shape)
# aka center per time point
move_td = subtract_cross_condition_mean(move_td)

pca_dims = 30
move_td = dim_reduce(move_td, PCA(pca_dims), "both_rates", "both_pca")
move_td["target_id"] = move_td.apply(get_target_id, axis=1)
av_per_dir = trial_average_fixed(move_td, "target_id")


max_iterations = 10000
output_dimension = pca_dims
cebra_model = CEBRA(model_architecture='offset10-model',
                        batch_size=256,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=output_dimension,
                        max_iterations=max_iterations,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

cebra_model.fit(processed_X_Test, processed_Y_Test)  #For creating test embedding
test_embed = cebra_model.transform(processed_X_Test)

for input_field in ["M1_rates", "PMd_rates", "both_rates", "both_pca"]:
    X = concat_trials(move_td, input_field)
    y = concat_trials(move_td, "vel")

    cv_scores = cross_val_score(LinearRegression(), X, y, cv = KFold(5, shuffle=True))

    print(f"{input_field}:\t{cv_scores.mean()}")